# [config.py](..\utils\config.py)

## import 导入链的注意事项

导入是将被导入模块中的名称添加到导入模块的命名空间中的过程。

假设当前模块名为 `M`，`from X import Xo` 会将 `Y` 直接添加到当前模块的命名空间：可以访问 `M.Xo`

而 `import X` 会将整个模块 `X` 作为一个对象添加到当前模块的命名空间：`M.X`（包括例如 `M.X.Xo`）等。

例子 1：

```python
# 模块A (a.py)
value = 100
def func(): 
    return "A.func"

# 模块B (b.py)
from A import value, func  # 从A导入特定内容到B的命名空间

# 模块C (c.py)
import B

# 在C中可以使用:
print(B.value)  # 可以访问，输出100
print(B.func())  # 可以访问，输出"A.func"
```
例子 2：
```python
# 模块A (a.py)
value = 100
def func(): 
    return "A.func"

# 模块B (b.py)
import A  # A成为B模块命名空间中的一个对象

# 模块C (c.py)
import B

# 在C中:
print(B.A.value)  # 可以访问，输出100
print(B.value)    # 错误! B中没有直接定义value
```
## 描述符
### 描述符协议

描述符协议规定了：`描述符` 作为某个属主类的 `类属性` 时，属主类使用该描述符时实际发生的行为。

一个类要成为 `描述符`，至少要实现下面三个特殊方法中的一个：

```python
__get__(self, instance, owner)
__set__(self, instance, value)
__delete__(self, instance)
```

其中 `instance` 是属主类实例，`owner` 是属主类本身。

```python
class Descriptor:

    def __init__(self):
        ...

    def __get__(self, instance, owner):
        ...

    def __set__(self, instance, value):
        ...

    def __delete__(self, instance):
        ...

class User:
    descriptor = Descriptor()

user = User()
```

当类似于 `= user.descriptor` 使用时，实际上是触发了 `descriptor.__get__(user, User)`。

`user.descriptor = 值` 实际上触发了 `descriptor.__set__(user, 值)`。

`del user.descriptor` 实际上触发了 `descriptor.__delete__(user)`。

### property 类

`property` 是一个实现了描述符协议的 Python 内置类，其实现：

```python
class Property:
    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc"

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)

    # 返回新的 Property 实例
    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)
```

### @property

使用方法如下

```python
class MyClass:
    def __init__(self):

    # value = property(函数 value(self))
    @property
    def value(self):
        ...

    # value = value.setter(函数 value(self, new_value))
    @value.setter
    def value(self, new_value):
        ...

    # value = value.setter(函数 value(self))
    @value.deleter
    def value(self):
        ...
```

所以相当于创建了三次 property 实例，最后该实例成为 MyClass 的一个类属性 value。

然后 MyClass 或其实例就可以按照 `描述符协议` 来使用 value。

## `__dict__` 和字典类的 `keys()`
### `__setattr__` 和 `__setitem__`
通过 `点表示法` 给实例的属性赋值时，会调用 `__setattr__`，默认情况下，会将该属性的值更新到该实例的 `__dict__`。

实现了 `__setitem__` 方法的类型，其实例使用 `[]` 索引法赋值时，会调用 `__setitem__`。
### `__dict__` 和字典类的 `keys()`

类型和实例都有 `__dict__`，其可以理解为一个字典。
实例的 `__dict__` 存储了实例属性（包括 `__init__` 中 `self.* =`为所有实例绑定的，以及动态绑定的），而类型的 `__dict__` 存储了类属性和方法。

字典 `dict` 有 `keys()`、`values()` 和 `items()` 方法。继承了字典的类自然也继承了这三个方法。
### Config 类的 `_as_attrs_`, `__setattr__`, `__setitem__`
注意到 `Config(PickleableDict, Documented)<——PickleableDict(Pickleable, dict)`，所以 `Config` 是字典类。

`Config` 作为字典类，以及 `_as_attrs_`, `_readonly_`, `_frozen_keys_`, `__setattr__`, `__setitem__` 的实现，构成了：
- `__dict__` 和 `dict` 是两个存储实例属性的结构；
- `_as_attrs_` 是 `false`：点赋值无效，只提供了索引赋值`dict`（且不能是`非强制更新 && _readonly_ || 非强制更新 && _frozen_keys_ && 索引的键在 dict 中` 的情况）；
- `_as_attrs_` 是 `true`：无论点赋值或索引赋值，`非强制更新 && _readonly_ || 非强制更新 && _frozen_keys_ && 索引的键在 dict 中` 决定了是否更新 `__dict__` 和 `dict`；

所以这种设计思想是：
- `self` 具有 `items()` 和 `__dict__` 两个属性存储结构。
- `_as_attrs_` 决定了是否支持点赋值
- 当实施赋值时，`_readonly_` 和 `_frozen_keys_` 决定了是否允许赋值。
- 允许赋值后，`_as_attrs_` 还决定了是否更新 `__dict__`。

## `with open(str, '*') as f:`
等价于
```python
try:
    f = open(str, '*')
    ...
finally:
    if f:
        f.close()
```

## `__new__(cls, *args, **kwargs)`

`__new__` 是**静态方法**，其主要作用是分配内存并创建实例本身，在 `__init__` 之前。
其必须返回一个类的实例，通常这个实例是通过调用父类的 `__new__` 方法创建的，例如：
```python
instance = super().__new__(cls)
```
如果父类 `super()` 直接继承自 `object` 或者没有重定义 `__new__`，那么 `super().__new__(cls)` 实际上就是 `object.__new__(cls)`，
否则 `super()` 在其自身的 `__new__` 方法中也应该类似于上面，诸如此类，直到抵达调用 `object.__new__(cls)`。
如果在 `__new__` 中没有返回一个实例，那么 `__init__` 方法将不会被自动调用。

重写 `__new__` 最常见的场景是实现单例模式：`__new__` 可以检查实例是否已存在，如果存在则返回现有实例，否则创建新实例。

### 为什么 `__new__` 是静态方法
例如
```python
class MyClass:
    # super().__new__(cls) 中的 cls 即 MyClass
    # 所以类似的，__new__(cls, *args, **kwargs) 中的 cls 应该是由 MyClass 子类传入的
    def __new__(cls, *args, **kwargs):
        instance = super().__new__(cls)
        return instance  # 必须返回实例

    def __init__(self, val1, val2="default"):
        self.attribute1 = val1
        self.attribute2 = val2
```

# [checks.py](..\utils\checks.py)

## 哈希协议与可哈希类型
一个类型如果实现了 `__hash__` 方法，即认为其实现了协议类型 `collections.abc.Hashable`。
由于任意类型都继承了 `object`，而 `object` 实现了 `__hash__`，即返回实例的内存地址，所以任意类型都实现了哈希协议。

但是实现了哈希协议并不意味着可哈希，可以通过执行`hash(obj)`来判断：
- Python查找 `obj.__hash__()` 方法
- 如果找到了该方法，则调用它并返回结果
- 如果没有找到该方法或方法返回`None`，则抛出`TypeError: unhashable type`异常

重写一个类型的 `__hash__` 的规范的方式是：如果 `__eq__` 判断两个实例相同，那么这两个实例 `__hash__` 的返回值也必须相同
（注意 `__eq__` 的默认实现是比较两个实例的内存地址是否相同）。

## 命名元组 `namedtuple`

`namedtuple` 是 Python 标准库 `collections` 模块中的一个工厂函数，主要目的是为了解决普通元组在访问元素时只能通过索引的问题，适合用于定义简单的记录或数据结构。

语法：`namedtuple(typename, field_names)`
- typename: 创建的 namedtuple 类的名称（字符串）。
- field_names: 一个包含字段名称的列表或字符串（字段名称用空格或逗号分隔）。

例子：
```python
from collections import namedtuple

# 定义一个名为 'Student' 的 namedtuple 类，包含 'name', 'age', 'DOB' 三个字段
Student = namedtuple('Student', ['name', 'age', 'DOB'])
# 创建一个 Student 实例
s1 = Student('Nandini', '19', '2541997')

# 访问元素
print(s1.name)  # 通过属性名访问
print(s1[1])    # 通过索引访问
```

## 断言

`断言(assert)` 的一般格式为 `assert <expression> [, <optional_message>]`。其中
- `<expression>`: 这是一个布尔表达式，assert 会对其进行求值
- `[, <optional_message>]`: 可选部分，一个字符串
- 如果 <expression> 求值为假，那么 <optional_message> 会被作为参数构造并抛出 `AssertionError` 错误，否则继续运行

例子：

```python
python err.py
def foo(s):
    n = int(s)
    assert n != 0, 'n is zero!'
    return 10 / n

foo('0')

# 结果
Traceback (most recent call last):
  ...
AssertionError: n is zero!
```

## `typing.Any` 参与逻辑运算

当 `typing.Any`（类型或实例）参与逻辑运算时
- 实例：
    - 特定类型的特定实例，例如：`None, False, 0, 0.0, [], (), {}, ''`等，会被视为假值
    - 如果类型定义了 `__bool__`，那么相当于 `——>bool(实例)——>实例.__bool__()`；
    - 如果类型没有定义 `__bool__` 但是定义了 `__len__`，那么会调用 `len(实例)` 也就是 `实例.__len__()`；
    - 其余情况，一般默认为真值；
- 类型：通常被认为是真值。

## `arg1 in arg2`（前面没有`for`）
如果 `arg2` 对应的类型定义了 `__contains__` 方法，那么 `arg1 in arg2` 相当于 `arg2.__contains__(arg1)`。

如果 `arg2` 对应的类型没有 `__contains__` 方法但是是可迭代对象即定义了 `__iter__`，那么相当于：
```python
for x in arg2
    if x == arg1
        return True
return False
```

# [docs.py](..\utils\docs.py)

## JSON 序列化
在不同的编程语言之间传递对象，必须把对象序列化为标准格式，比较好的方法是序列化为 `JSON`。

因为 `JSON` 表示出来就是一个字符串，可以被所有语言读取，也可以方便地存储到磁盘或者通过网络传输。

### JSON类型

`JSON` 和 Python 内置的数据类型对应如下：

| JSON类型   | Python类型 |
| :--------- | :--------- |
| {}         | dict       |
| []         | list       |
| "string"   | str        |
| 1234.56    | int或float |
| true/false | True/False |
| null       | None       |

### 自定义类型实例的JSON化

很多时候，希望把自定义类的实例 `JSON` 序列化：

```python
import json

class Student(object):
    def __init__(self, name, age, score):
        self.name = name
        self.age = age
        self.score = score

s = Student('Bob', 20, 88)
print(json.dumps(s))

# 结果
Traceback (most recent call last):
  ...
TypeError: <__main__.Student object at 0x10603cc50> is not JSON serializable
```

错误的原因是当前 `Student` 对象不是一个可序列化为 `JSON` 的对象。

`dumps()` 方法的参数列表，除了第一个必须的`obj`参数外，还有很多可选参数：

[https://docs.python.org/3/library/json.html#json.dumps](https://docs.python.org/3/library/json.html#json.dumps)

可以通过这些可选参数来定制 `JSON` 序列化。

**前面的 `Student` 实例无法 `JSON` 序列化，是因为默认情况下 `dumps()` 不知道如何将 `Student` 实例变为一个 JSON 的 `{}` 对象**。

可选参数 `default` 就是把任意一个对象变成一个可序列为 `JSON` 的对象，只需要为 `Student` 写一个转换函数：

```python
def student2dict(std):
    return {
        'name': std.name,
        'age': std.age,
        'score': std.score
    }

# Student 实例首先被 student2dict() 函数转换成 dict ，然后再被序列化为 JSON
>>> print(json.dumps(s, default=student2dict))
{"age": 20, "name": "Bob", "score": 88}
```

一种可以将任意类的实例变成可 `JSON` 序列化的方式：

```python
# 利用 lambda 表达式，直接传递实例的 __dict__ 属性
print(json.dumps(s, default=lambda obj: obj.__dict__))
```

同样，如果要把 `JSON` 反序列化为一个实例，`loads()` 方法首先转换出一个 `dict` 对象；

然后，我们传入的 `object_hook` 函数负责把 `dict` 转换为实例：

```python
def dict2student(d):
    return Student(d['name'], d['age'], d['score'])

>>> json_str = '{"age": 20, "score": 88, "name": "Bob"}'
>>> print(json.loads(json_str, object_hook=dict2student))
<__main__.Student object at 0x10cd3c190>
```